In [1]:
# MeasureNoteModel과 MeasureNumberSet을 사용하는 경우를 분석하려고 한다.
# MeasureNumberSet은 get_item의 경우에 3가지 아웃풋을 내보낸다.
# melody, shifted_melody, measure_numbers의 아웃풋 중에서 
# melody와 measure_numbers만을 모델에 넣어주면 된다.
# shifted_melody는 melody를 한 칸씩 shift한 것으로 모델의 pred와 비교해 loss를 구할때 사용한다.

In [4]:
# trainer.py에서 batch를 받아서 model에 넣어주는 과정에서 3등분을 해준다.
# melody, shifted_melody, measure_numbers = batch
# collate_fn을 거쳐 각각은 packed sequence로 만들어진다.
# 따라서 인풋은 token_size x input_size의 텐서로 변환된다.
# MeasureNumberSet의 경우에는 input_size가 20이다.
# 그 종류는 아래와 같다.
['main', 'dur', 'pitch_class', 'octave', 'm_idx', 'm_idx_mod4', 'm_offset', 'is_onbeat', 'is_middle_beat',
 'key', 'meter', 'unit_length', 'rhythm', 'root', 'mode', 'key_sig',  'numer', 'denom','is_compound', 'is_triple']

['main',
 'dur',
 'pitch_class',
 'octave',
 'm_idx',
 'm_idx_mod4',
 'm_offset',
 'is_onbeat',
 'is_middle_beat',
 'key',
 'meter',
 'unit_length',
 'rhythm',
 'root',
 'mode',
 'key_sig',
 'numer',
 'denom',
 'is_compound',
 'is_triple']

모델의 구조에서 forward 부분을 살펴볼 필요가 있다.
순서는 input_seq -> embedding layer -> rnn layer -> measure_rnn -> final_rnn -> projection(linear) layer로 이루어져 있다.

In [1]:
def forward(self, input_seq, measure_numbers):
    if isinstance(input_seq, PackedSequence):
      emb = self._get_embedding(input_seq)
      hidden, _ = self.rnn(emb)
      measure_hidden = self.measure_rnn(hidden, measure_numbers)

      cat_hidden = PackedSequence(torch.cat([hidden.data, measure_hidden.data], dim=-1), hidden.batch_sizes, hidden.sorted_indices, hidden.unsorted_indices)
      
      final_hidden, _ = self.final_rnn(cat_hidden)

      logit = self.proj(final_hidden.data)
      prob = self._apply_softmax(logit)
      prob = PackedSequence(prob, input_seq[1], input_seq[2], input_seq[3])
      return prob
    else:
      raise NotImplementedError

In [3]:
## embedding
# model.emb에 들어가는 것은 input 중에서 melody이다.
# data_sample = next(iter(train_loader))
# emb = model._get_embedding(data_sample[0])
# data_sample[0].data.shape
# torch.Size([4820, 20])
# emb.data.shape
# torch.Size([4820, 2624])

In [2]:
## rnn
# rnn에서 헷갈리기 쉬운 것은 hidden과 last_hidden이다.
# hidden은 token(sum of timestep of input sequence in every batch) x embedding_size(2624)를 token x hidden_size(512)로 변환해준다.
# last_hidden은 마지막 token이 가질 모든 정보를 담고 있다. num_layers(3) x num_directions(1) x batch_size(32) x hidden_size(512)
# hidden, last_hidden = model.rnn(emb)
# hidden.data.shape
# torch.Size([4820, 512])
# last_hidden.data.shape
# torch.Size([3, 32, 512])

In [3]:
## measure_rnn
# attention을 사용한다. 명확하게 이해하지는 못했다.
# input 중에서 3번째인 measure_numbers를 함께 넣어준다.
# measure_hidden = model.measure_rnn(hidden, data_sample[2])
# measure_hidden.data.shape
# torch.Size([4820, 512])

In [1]:
## final_rnn
# cat_hidden은 hidden과 measure_hidden의 data 부분을 concat한 것이다.
# 이로써 각 토큰은 1024차원의 latent vector를 갖게 된다.
# 이 latent vector를 final_rnn에 넣어준다.
# final_hidden, last_final_hidden = model.final_rnn(cat_hidden)
# final_hidden.data.shape
# torch.Size([4820, 512])
# last_final_hidden.data.shape
# torch.Size([3, 32, 512])

이후에 title embedding을 위해서는 각각의 tune에 맞는 embedding vector를 구해야한다.
이를 위해서 나는 final_hidden을 학습하는 추가적인 rnn을 만들거나 attention을 구현해야 한다.
우선은 bi-directional rnn을 사용해보려고 한다.
rnn이 순차적으로 모든 token을 양방향으로 보고 hidden vector를 token x vector로  